# Repertoire

In [ ]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
except:
  print("vous n'êtes encore  sur Google Colab")

Mounted at /gdrive


In [ ]:
REP_PROJET = '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
REP_INPUT  = REP_PROJET + 'Input/'
REP_INTERMED  = REP_PROJET + 'Intermed/'
REP_OUTPUT  = REP_PROJET + 'Output/'

# MODULES

In [ ]:
%cd '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
!pwd

/gdrive/My Drive/Colab Notebooks/Dedoublement
/gdrive/My Drive/Colab Notebooks/Dedoublement


In [ ]:
%%capture
pip install -r requirements.txt

In [ ]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import numpy as np
import re

# FUNCTIONS

In [ ]:
%run "00_functions.ipynb"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Discogs data

## Load 

In [ ]:
baseline = pd.read_pickle(REP_INTERMED+ "baseline.sav")

## Vectorizer :  Keep TF-IDF strategy 

In [ ]:
#start_time = datetime.now()

# create_vectorizer_and_vectorized_db(baseline, analyzer = 'word',ngram_range=(1,1))

#vend_time = datetime.now()

#print('Duration: {}'.format(end_time - start_time))

#Duration: 02 min:15s

In [ ]:
X_train, vectorizer = reload_data_and_vectorizer(analyzer = 'word',ngram_range=(1,1))

# Query Data

In [ ]:
df_no_match = pd.read_pickle(REP_INTERMED +'base_cdandlp.sav')

df_sample = df_no_match.sample(6, random_state=2022).reset_index(drop=True)

df_sample['text'] = df_sample['artiste'] +  ' ' +df_sample['titre']

# KNN unsupervised (sklearn api)

## Training with Manahattan

https://github.com/scikit-learn/scikit-learn/blob/86b1ba72771718acbd1e07fbdc5caaf65ae65440/sklearn/neighbors/unsupervised.py#L48

In [ ]:
analyzer = 'char'

ngram_range = (1,1)
suffix_gram = re.sub(r"[(,)]", "", str(ngram_range)).replace(' ','_')
path = REP_INTERMED + analyzer + '_' + suffix_gram+  '_results_sim.pkl'

nbrs = NearestNeighbors(n_neighbors=200, metric= 'cosine',algorithm='brute').fit(X_train)

start_time = datetime.now()

result = [ nbrs.kneighbors(text_to_vect(df_sample.loc[i,'text'], vectorizer)) for i in range (len(df_sample)) ]

f = open(path, 'wb')

pickle.dump(result, f) 

f.close()

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

# Duration: 0:02:03.554452
# Duration: 0:02:03.554452


Duration: 0:00:24.720320


In [ ]:

  # enrcih the dataframe
indexes = indices.flatten().tolist()
baseline['index'] = baseline.index
neighbors_sklearn_api= pd.merge(
                                  baseline[baseline['index'].isin(indexes)], temp,
                                  on = 'index',
                                  how ='inner'
                                  ).\
                          sort_values(by='distance')

In [ ]:
colonnes = ['levenshtein', 'jaro_winkler', 'jaccard', 'overlap', 'hamming', 'fuzzy_partial']
column_order = ['id_query', 'master_id', 'artist', 'title', 'distance','rank']

In [ ]:
column_order + colonnes 

['id_query',
 'master_id',
 'artist',
 'title',
 'distance',
 'rank',
 'levenshtein',
 'jaro_winkler',
 'jaccard',
 'overlap',
 'hamming',
 'fuzzy_partial']

In [ ]:
neighbors_sklearn_api.apply(lambda p:compound_similarity(p, "text_CLEAN",  "text_query"), axis=1)

161    [0.30434782608695654, 0.44605475040257647, 0.2...
184    [0.3125, 0.42622834924001407, 0.18518518518518...
100    [0.3673469387755102, 0.5777591973244147, 0.324...
10     [0.3050847457627119, 0.5230043708304578, 0.340...
112    [0.2909090909090909, 0.528950668896321, 0.3095...
                             ...                        
51     [0.2857142857142857, 0.4826526130873956, 0.244...
52     [0.2962962962962963, 0.5251675237649992, 0.384...
53     [0.2465753424657534, 0.4802006688963212, 0.280...
56     [0.3111111111111111, 0.4964426877470356, 0.323...
0      [0.22727272727272727, 0.4584771106510237, 0.29...
Length: 200, dtype: object

# Customiz Output

In [ ]:
with open(path, 'rb') as file:
      result = pickle.load(file)

In [ ]:
temp = [ customize_output(i,results= result, df_baseline=baseline.copy(), sample=df_sample) for i in range(len(df_sample)) ]

In [ ]:
pip install fuzzywuzzy

In [ ]:
temp

[      id_query  master_id                          artist  \
 0    115969488  115969488                        Crasnaru   
 1    115969488          0  Псой Короленко, Алёна Аленкова   
 2    115969488          0  Псой Короленко, Алёна Аленкова   
 3    115969488    1582267  Псой Короленко И Опа Новый Год   
 4    115969488          0                             קוב   
 ..         ...        ...                             ...   
 196  115969488    1819226                    Rudán József   
 197  115969488    1819225                    Rudán József   
 198  115969488          0                    Rudán József   
 199  115969488          0                           Rudye   
 200  115969488          0                         Rue (7)   
 
                                            title  distance rank levenshtein  \
 0                                    Opera Arias      -1.0    0           1   
 1                       Русское Богатство. Том I  0.292893    1    0.285714   
 2            

In [ ]:
pd.concat(temp,axis=0).\
    rename(columns={'master_id':'id_text'}).\
    to_excel(REP_INTERMED + analyzer + '_' + suffix_gram  +'_neighbors_sklearn_api.xlsx', index= False)

,id_query,id_text,artist,title,distance,rank,levenshtein,jaro_winkler,jaccard,overlap,hamming,fuzzy_partial
0,115969488,115969488,Crasnaru,Opera Arias,-1.0,0,1,1,1,1,1,1
1,115969488,0,"Псой Короленко, Алёна Аленкова",Русское Богатство. Том I,0.292893,1,0.285714,0.506293,0.235294,0.421053,0.043478,0.32
2,115969488,0,"Псой Короленко, Алёна Аленкова",Русское Богатство. Том 2,0.292893,1,0.388889,0.593625,0.333333,0.529412,0.157895,0.41
3,115969488,1582267,Псой Короленко И Опа Новый Год,Под Покровом Нощи,0.292893,1,0.305085,0.576566,0.311111,0.736842,0.05,0.42
4,115969488,0,קוב,הזמן מזיע,0.42265,4,0.26087,0.489116,0.210526,0.421053,0.074074,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...
194,117795850,0,Die Fidelen Mölltaler,Du Allein,0.465478,131,0.327273,0.52122,0.341463,0.538462,0.034483,0.31
195,117795850,0,Lenny (2),Homies & Huren,0.465478,131,0.292683,0.515218,0.261538,0.653846,0.017857,0.38
196,117795850,0,Lenny (2),Hier,0.465478,131,0.25,0.439316,0.244444,0.423077,0.0,0.23
197,117795850,1587866,Die Fidelen Inntaler,Aus Deutschen Landern (German Country Dances),0.465478,131,0.266667,0.499246,0.333333,0.576923,0.029412,0.27


In [ ]:
a=pd.concat(temp,axis=0).\
    rename(columns={'master_id':'id_text'})

In [ ]:
a[a['rank'].astype(int) < 3].head(60)

,id_query,id_text,artist,title,distance,rank,levenshtein,jaro_winkler,jaccard,overlap,hamming,fuzzy_partial
0,115969488,115969488,Crasnaru,Opera Arias,-1.0,0,1,1,1,1,1,1
1,115969488,0,"Псой Короленко, Алёна Аленкова",Русское Богатство. Том I,0.292893,1,0.285714,0.506293,0.235294,0.421053,0.043478,0.32
2,115969488,0,"Псой Короленко, Алёна Аленкова",Русское Богатство. Том 2,0.292893,1,0.388889,0.593625,0.333333,0.529412,0.157895,0.41
3,115969488,1582267,Псой Короленко И Опа Новый Год,Под Покровом Нощи,0.292893,1,0.305085,0.576566,0.311111,0.736842,0.05,0.42
0,115111082,115111082,Pascal Bacoux,Confession,-1.0,0,1,1,1,1,1,1
1,115111082,1948642,Patricia Spero,Winter Harp,0.0,1,0.282051,0.523604,0.238095,0.652174,0.018182,0.35
2,115111082,0,Patricia Williams (2),"Law, Culture And Medical Ethics",0.0,1,0.326531,0.522204,0.4,0.608696,0.038462,0.35
0,115869550,115869550,Alain Souchon,C'Est Déjà Ça,-1.0,0,1,1,1,1,1,1
1,115869550,1298344,"Alan Walker (9), Noah Cyrus & Digital Farm Ani...",All Falls Down (Remixes),0.25,1,0.2,0.377442,0.162791,0.291667,0.076923,0.21
2,115869550,1523332,"Alan Walker (9), Sabrina Carpenter & Farruko",On My Way (Da Tweekaz Remix),0.25,1,0.282353,0.445241,0.180556,0.5,0.050847,0.38
